In [1]:
import numpy as np
import librosa
import soundfile as sf
from sklearn.decomposition import NMF

In [2]:
speech = "example/problem2/speech.wav"
audio, sr = librosa.load(speech, sr=None)
spectrogram = librosa.stft(audio, n_fft=2048, hop_length=256, center=False, win_length=2048)
Ms = abs(spectrogram)
phase_s = spectrogram/(Ms+2.2204e-16)

music = "example/problem2/music.wav"
audio, sr = librosa.load(music, sr=None)
spectrogram = librosa.stft(audio, n_fft=2048, hop_length=256, center=False, win_length=2048)
Mm = abs(spectrogram)
phase_m = spectrogram/(Mm+2.2204e-16)

music = "example/problem2/mixed.wav"
audio, sr = librosa.load(music, sr=None)
spectrogram = librosa.stft(audio, n_fft=2048, hop_length=256, center=False, win_length=2048)
Mmixed = abs(spectrogram)
phase_mixed = spectrogram/(Mmixed+2.2204e-16)

In [41]:
speech_model = NMF(n_components=300, init='random', random_state=0, solver='mu')
music_model = NMF(n_components=300, init='random', random_state=0, solver='mu')

Bs = speech_model.fit_transform(Ms)
Bm = speech_model.fit_transform(Mm)

sources = np.concatenate((Bs,Bm), axis=1)

/Users/donghyunlee/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
/Users/donghyunlee/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [44]:
H = np.random.rand(600,976).astype(dtype=np.float32)

In [45]:
mixed_model = NMF(n_components=600, init='custom',regularization='components', solver='mu')
output = mixed_model.fit_transform(Mmixed, W=sources, H=H)

(1025, 600)


In [47]:
W_mixed = mixed_model.components_
W_speech_pred = W_mixed[:300,:]
W_music_pred = W_mixed[300:,:]

M_speech_pred = np.dot(Bs, W_speech_pred)
M_music_pred = np.dot(Bm, W_music_pred)

[[4.9955543e-15 8.0589113e-21 9.3510287e-17 ... 2.2200603e-02
  2.5503108e-02 7.4445203e-02]
 [1.6767317e-03 4.3087029e-03 3.6127962e-02 ... 4.2515479e-02
  6.7156248e-02 1.7579522e-02]
 [1.4396367e-03 1.5756381e-01 4.0908400e-03 ... 1.8842620e-03
  4.0630240e-02 5.4099310e-02]
 ...
 [8.8359192e-02 2.8497696e-01 4.3086845e-01 ... 2.3562914e-02
  2.0240063e-03 4.3486343e-09]
 [1.2830082e-01 2.7191710e-01 3.4009837e-02 ... 1.1988609e-01
  1.9305778e-01 9.3056746e-02]
 [9.3106609e-03 3.1268265e-04 2.4829980e-02 ... 4.5669448e-01
  5.5226761e-01 5.0026506e-01]]


In [48]:
signal_hat = librosa.istft(M_speech_pred*phase_mixed, hop_length=256, center=False, win_length=1024)
sf.write("example/speech_rec.wav", signal_hat, samplerate=16000)
signal_hat = librosa.istft(M_music_pred*phase_mixed, hop_length=256, center=False, win_length=1024)
sf.write("example/music_rec.wav", signal_hat, samplerate=16000)